<a href="https://colab.research.google.com/github/Pranjal1092/OASIS/blob/main/text_based_chat_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Server Code

In [ ]:
import socket
import threading

# Function to handle client connection
def handle_client(client_socket, client_address, clients):
    print(f"New connection: {client_address}")
    while True:
        try:
            message = client_socket.recv(1024).decode('utf-8')
            if message:
                print(f"{client_address}: {message}")
                broadcast(message, client_socket, clients)
            else:
                remove_client(client_socket, clients)
                break
        except:
            continue

# Function to broadcast a message to all clients
def broadcast(message, sender_socket, clients):
    for client in clients:
        if client != sender_socket:
            try:
                client.send(message.encode('utf-8'))
            except:
                remove_client(client, clients)

# Function to remove a client from the list
def remove_client(client_socket, clients):
    if client_socket in clients:
        clients.remove(client_socket)
        client_socket.close()

def main():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('0.0.0.0', 12345))
    server_socket.listen(5)
    print("Server started on port 12345")

    clients = []

    while True:
        client_socket, client_address = server_socket.accept()
        clients.append(client_socket)
        client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address, clients))
        client_thread.start()

if __name__ == "__main__":
    main()


Server started on port 12345


Client Code

In [ ]:
import socket
import threading

# Function to handle receiving messages from the server
def receive_messages(client_socket):
    while True:
        try:
            message = client_socket.recv(1024).decode('utf-8')
            if message:
                print(message)
        except:
            print("An error occurred!")
            client_socket.close()
            break

def main():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = input("Enter server IP address: ")
    client_socket.connect((server_address, 12345))

    receive_thread = threading.Thread(target=receive_messages, args=(client_socket,))
    receive_thread.start()

    while True:
        message = input()
        if message.lower() == 'exit':
            client_socket.close()
            break
        client_socket.send(message.encode('utf-8'))

if __name__ == "__main__":
    main()
